# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとCSVファイルからのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [12]:
# %load_ext sql
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

# pgconfig = {
#     'host': 'db',
#     'port': os.environ['PG_PORT'],
#     'database': os.environ['PG_DATABASE'],
#     'user': os.environ['PG_USER'],
#     'password': os.environ['PG_PASSWORD'],
# }
# dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# # pd.read_sql用のコネクタ
# conn = psycopg2.connect(**pgconfig)
# # pd.to_sql用のcreate engine
# engine = create_engine(dsl)
# # MagicコマンドでSQLを書くための設定
# %sql $dsl

# df_customer = pd.read_sql(sql='select * from customer', con=conn)
# df_category = pd.read_sql(sql='select * from category', con=conn)
# df_product = pd.read_sql(sql='select * from product', con=conn)
# df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
# df_store = pd.read_sql(sql='select * from store', con=conn)
# df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\iiats\Anaconda3\envs\Python36-gpu\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [14]:
df_category.head()

,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
0,4,惣菜,401,御飯類,40101,弁当類
1,4,惣菜,401,御飯類,40102,寿司類
2,4,惣菜,402,佃煮類,40201,魚介佃煮類
3,4,惣菜,402,佃煮類,40202,海草佃煮類
4,4,惣菜,402,佃煮類,40203,野菜佃煮類


In [15]:
df_product.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0
3,P040101004,4,401,40101,248.0,186.0
4,P040101005,4,401,40101,268.0,201.0


In [16]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [17]:
df_store.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


In [18]:
df_geocode.head()

,postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
0,060-0000,北海道,札幌市中央区,NaN,NaN,NaN,北海道札幌市中央区,141.34103,43.05513
1,064-0941,北海道,札幌市中央区,旭ケ丘,NaN,NaN,北海道札幌市中央区旭ケ丘,141.31972,43.04223
2,060-0042,北海道,札幌市中央区,大通西,NaN,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102
3,060-0042,北海道,札幌市中央区,大通西,NaN,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080
4,060-0042,北海道,札幌市中央区,大通西,NaN,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086


# 演習問題

---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [57]:
pd.merge(df_receipt, df_store[['store_cd','store_name']], how='inner', on='store_cd').head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1258329600,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1200614400,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1274659200,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1271635200,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
5,20181119,1258588800,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店
6,20171211,1228953600,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店
7,20191021,1287619200,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店
8,20170710,1215648000,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店
9,20190805,1280966400,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店


---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [58]:
pd.merge(df_product
         , df_category[['category_major_cd', 'category_medium_cd','category_small_cd','category_small_name']]
         , how='inner', on=['category_major_cd', 'category_medium_cd','category_small_cd']).head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,4,401,40101,198.0,149.0,弁当類
1,P040101002,4,401,40101,218.0,164.0,弁当類
2,P040101003,4,401,40101,230.0,173.0,弁当類
3,P040101004,4,401,40101,248.0,186.0,弁当類
4,P040101005,4,401,40101,268.0,201.0,弁当類
5,P040101006,4,401,40101,298.0,224.0,弁当類
6,P040101007,4,401,40101,338.0,254.0,弁当類
7,P040101008,4,401,40101,420.0,315.0,弁当類
8,P040101009,4,401,40101,498.0,374.0,弁当類
9,P040101010,4,401,40101,580.0,435.0,弁当類


---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [59]:
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_tmp = df_customer.query('gender_cd == "1" and not customer_id.str.startswith("Z")', engine='python')
pd.merge(df_tmp['customer_id'], df_amount_sum, how='left', on='customer_id').fillna(0).head(10)

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [60]:
df_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_sum = df_sum.query('not customer_id.str.startswith("Z")', engine='python')
df_sum = df_sum.sort_values('amount', ascending=False).head(20)

df_cnt = df_receipt[~df_receipt.duplicated(subset=['customer_id', 'sales_ymd'])]
df_cnt = df_cnt.query('not customer_id.str.startswith("Z")', engine='python')
df_cnt = df_cnt.groupby('customer_id').sales_ymd.count().reset_index()
df_cnt = df_cnt.sort_values('sales_ymd', ascending=False).head(20)

pd.merge(df_sum, df_cnt, how='outer', on='customer_id')

,customer_id,amount,sales_ymd
0,CS017415000097,23086.0,20.0
1,CS015415000185,20153.0,22.0
2,CS031414000051,19202.0,19.0
3,CS028415000007,19127.0,21.0
4,CS001605000009,18925.0,NaN
5,CS010214000010,18585.0,22.0
6,CS016415000141,18372.0,20.0
7,CS006515000023,18372.0,NaN
8,CS011414000106,18338.0,NaN
9,CS038415000104,17847.0,NaN


---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [61]:
df_store_tmp = df_store.copy()
df_product_tmp = df_product.copy()

df_store_tmp['key'] = 0
df_product_tmp['key'] = 0
len(pd.merge(df_store_tmp, df_product_tmp, how='outer', on='key'))

531590